In [14]:
import re
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [2]:
def stoicoeff(x):
    if '*' in x:
        stoi = (x.split('*')[1], int(x.split('*')[0]))
    else:
        stoi = (x, 1)
    return stoi

def rate(conc, rxnlist, cmplist):
    c = {cmp: 0 for cmp in cmplist}
    rrate = [0 for r in rxnlist]
    for (i, cmp) in enumerate(cmplist):
        c[cmp] = conc[i]
    for (i, r) in enumerate(rxnlist):
        rf = r['p']['kf']
        for (cmp, n) in r['info'][0].items():
            rf = rf * c[cmp] ** n
        if r['info'][2] == 'eq':
            rr = kr
            for (cmp, n) in r['info'][1].items():
                rr = rr * c[cmp] ** n
        else:
            rr = 0.0 
        rrate[i] = rf - rr
    dc = {cmp: 0 for cmp in cmplist}
    for (i, r) in enumerate(rxnlist):
        for cmp in cmplist:
            if cmp in r['info'][0]:
                dc[cmp] = dc[cmp] - r['info'][0][cmp] * rrate[i]
            elif cmp in r['info'][1]:
                dc[cmp] = dc[cmp] + r['info'][1][cmp] * rrate[i] 
    dconc = [0 for cmp in cmplist]
    for (i, cmp) in enumerate(cmplist):
        dconc[i] = dc[cmp]
    
    return dconc
    

In [3]:
rxnlist = [{'r': 'A -> B', 'p': {'kf':1, 'Keq': 1000, 'Eaf': 0, 'Ear': 0}}, 
           {'r': 'B -> C', 'p': {'kf':1, 'Keq': 1000, 'Eaf': 0, 'Ear': 0}}]

for rxn in rxnlist:
    rxn2 = re.split('->|=', rxn['r'])
    rxntype = 'eqb' if '=' in rxn['r'] else 'fwd'
    reactants = [x.strip() for x in rxn2[0].split('+')]
    products = [x.strip() for x in rxn2[1].split('+')]
    stoi_reactants = {stoicoeff(x)[0]: stoicoeff(x)[1] for x in reactants}
    stoi_products = {stoicoeff(x)[0]: stoicoeff(x)[1] for x in products}
    rxn['info'] = stoi_reactants, stoi_products, rxntype
    
rxnlist

[{'r': 'A -> B',
  'p': {'kf': 1, 'Keq': 1000, 'Eaf': 0, 'Ear': 0},
  'info': ({'A': 1}, {'B': 1}, 'fwd')},
 {'r': 'B -> C',
  'p': {'kf': 1, 'Keq': 1000, 'Eaf': 0, 'Ear': 0},
  'info': ({'B': 1}, {'C': 1}, 'fwd')}]

In [4]:
data_df = pd.read_csv("ABC_data.csv")
data_df.head()
# Convert data to a list of dictionaries
data = [{'ca_meas': {k:v for (k, v) in zip(data_df.t, data_df.ca)},
    'cb_meas': {k:v for (k, v) in zip(data_df.t, data_df.cb)},
    'cc_meas': {k:v for (k, v) in zip(data_df.t, data_df.cc)} }]
t_meas = data[0]['ca_meas'].keys()

In [31]:
cmplist = ['A', 'B', 'C']

def ssq(beta, data, rxnlist, cmplist):
    rxnlist[0]['p']['kf'] = beta[0]
    rxnlist[1]['p']['kf'] = beta[1]
    
    #print(rxnlist)
    
    rate_lam = lambda t, c: rate(c, rxnlist, cmplist)
    
    csol = solve_ivp(rate_lam, [0, 10], [1.0, 0.0, 0.0], method = 'BDF', t_eval = list(t_meas))
    ca_pred = csol.y[0]
    cb_pred = csol.y[1]
    cc_pred = csol.y[2]
    
    ca_meas = list(data[0]['ca_meas'].values())
    cb_meas = list(data[0]['cb_meas'].values())
    cc_meas = list(data[0]['cc_meas'].values())
    ssq = np.sum((ca_pred - ca_meas)**2) + np.sum((cb_pred - cb_meas)**2) + np.sum((cc_pred - cc_meas)**2)
    
    return ssq
    

In [33]:
ssq(np.array([1.0, 1.0]), data, rxnlist, cmplist)

0.6530175180488677

In [36]:
res = minimize(ssq, np.array([1.0, 1.0]), args = (data, rxnlist, cmplist), method = 'BFGS')

In [37]:
res

      fun: 0.0260833272925221
 hess_inv: array([[ 2.7173801 , -0.40426801],
       [-0.40426801,  0.3349931 ]])
      jac: array([ 8.07922333e-08, -3.61818820e-07])
  message: 'Optimization terminated successfully.'
     nfev: 36
      nit: 7
     njev: 9
   status: 0
  success: True
        x: array([2.01846548, 0.99337053])